In [36]:
import os
import dotenv
import pandas as pd
from crewai import Agent, Task, Crew, Process,LLM
from pydantic import BaseModel, Field
from typing import Literal


In [34]:
dotenv.load_dotenv()

True

In [ ]:
os.chdir(r'C:\work\crewai_customer_support_chatbot\deep_resarch')

'C:\\work\\crewai_customer_support_chatbot\\deep_resarch'

In [13]:
uncleaned_dataset = pd.read_csv(r'C:\work\crewai_customer_support_chatbot\deep_resarch\data\customer_support_tickets.csv')

In [14]:
df = uncleaned_dataset.copy()

In [15]:
df.columns = df.columns.str.lower().str.replace(' ', '_')

In [16]:
df = df[df['ticket_status']=='Closed']

In [59]:
df['ticket_description'] = df.apply(
    lambda row: row['ticket_description'].lower().replace('product_purchased', str(row['product_purchased'])),
    axis=1
)

In [60]:
df['ticket_type'] = df['ticket_type'].str.lower().str.replace(' ', '_')
df['ticket_subject'] = df['ticket_subject'].str.lower().str.replace(' ', '_')

In [61]:
unique_ticket_types = df['ticket_type'].unique().tolist()
unique_ticket_subset = df['ticket_subject'].unique().tolist()

In [62]:
from typing import Literal
from pydantic import BaseModel, Field

class TicketType(BaseModel):
    ticket_type: Literal[*unique_ticket_types] = Field(..., description="The type of ticket")
    ticket_subject: Literal[*unique_ticket_subset] = Field(..., description="The subset of ticket")
    reasoning: str = Field(..., description="Brief explanation for the classification")



In [63]:
# Get unique ticket types and subjects for classification
print("Unique Ticket Types:")
print(unique_ticket_types)
print("\nUnique Ticket Subjects (first 10):")
print(unique_ticket_subset)


Unique Ticket Types:
['technical_issue', 'billing_inquiry', 'cancellation_request', 'product_inquiry', 'refund_request']

Unique Ticket Subjects (first 10):
['network_problem', 'account_access', 'data_loss', 'software_bug', 'product_setup', 'product_recommendation', 'cancellation_request', 'product_compatibility', 'hardware_issue', 'delivery_problem', 'payment_issue', 'refund_request', 'display_issue', 'battery_life', 'installation_support', 'peripheral_compatibility']


In [64]:
llm_model = LLM(
    model=os.getenv("MODEL"),
    temperature=0.0
)

In [65]:
# Create the Ticket Classification Agent
ticket_classifier_agent = Agent(
    role='Customer Support Ticket Classifier',
    goal='Accurately classify customer support ticket descriptions into the correct ticket type',
    backstory="""You are an expert customer support analyst with years of experience 
    in categorizing and triaging customer support tickets. You have deep understanding 
    of different types of customer issues including technical problems, billing inquiries, 
    cancellation requests, product inquiries, and refund requests. You analyze ticket 
    descriptions carefully and classify them accurately based on the customer's actual need.""",
    verbose=True,
    allow_delegation=False,
    llm=llm_model
)


In [66]:
# Create the Classification Task
def create_classification_task(ticket_description: str):
    """Create a task to classify a ticket description"""
    
    # Create a formatted list of possible ticket types
    ticket_types_list = "\n".join([f"- {tt}" for tt in unique_ticket_types])
    ticket_subset_list = "\n".join([f"- {tt}" for tt in unique_ticket_subset])
    
    
    task = Task(
        description=f"""Analyze the following customer support ticket description and classify it 
        into one of the available ticket types.
        
        Ticket Description:
        {ticket_description}
        
        Available Ticket Types:
        {ticket_types_list}

        Available Ticket subject:
        {ticket_subset_list}
        
        Carefully read the ticket description and determine which ticket type best matches 
        the customer's issue or request. Consider the main intent and primary concern of the customer.
        """,
        expected_output="""A JSON object with the following structure:
        {{
            "ticket_type": "the classified ticket type (must be one from the available types)",
            "ticket_subject": "the classified ticket subject (must be one from the available subjects)",
            "reasoning": "brief explanation of why you chose this classification"
        }}""",
        agent=ticket_classifier_agent,
        output_pydantic=TicketType
    )
    
    return task


In [67]:
# Function to classify a ticket
def classify_ticket(ticket_description: str):
    """Classify a single ticket description"""
    
    # Create task for this ticket
    classification_task = create_classification_task(ticket_description)
    
    # Create crew
    crew = Crew(
        agents=[ticket_classifier_agent],
        tasks=[classification_task],
        verbose=False
    )
    
    # Run the crew
    result = crew.kickoff()
    
    return result.pydantic


In [78]:
# Test with a sample ticket description
sample_ticket = df.iloc[0]['ticket_description']
print("Sample Ticket Description:")
print(sample_ticket)
print("\n" + "="*80 + "\n")

# Classify the ticket
result = classify_ticket(sample_ticket)
print("\n" + "="*80 + "\n")
print("Classification Result:")
print(result)

Sample Ticket Description:
i'm facing a problem with my {Dell XPS}. the {Dell XPS} is not turning on. it was working fine until yesterday, but now it doesn't respond.

1.8.3 i really i'm using the original charger that came with my {Dell XPS}, but it's not charging properly.




╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Customer Support Ticket Classifier                                                                      │
│                                                                                                                 │
│  Task: Analyze the following customer support ticket description and classify it                                │
│          into one of the available ticket types.                                                                │
│                                                                                                                 │
│          Ticket Description:                                                                                    │
│          i'm facing a problem with my {Dell XPS}. the {Dell XPS} is not turning on. it was working fine until   │
│  yesterday, but now it doesn't respond.                                                                         │
│                                                                                                                 │
│  1.8.3 i really i'm using the original charger that came with my {Dell XPS}, but it's not charging properly.    │
│                                                                                                                 │
│          Available Ticket Types:                                                                                │
│          - technical_issue                                                                                      │
│  - billing_inquiry                                                                                              │
│  - cancellation_request                                                                                         │
│  - product_inquiry                                                                                              │
│  - refund_request                                                                                               │
│                                                                                                                 │
│          Available Ticket subject:                                                                              │
│          - network_problem                                                                                      │
│  - account_access                                                                                               │
│  - data_loss                                                                                                    │
│  - software_bug                                                                                                 │
│  - product_setup                                                                                                │
│  - product_recommendation                                                                                       │
│  - cancellation_request                                                                                         │
│  - product_compatibility                                                                                        │
│  - hardware_issue                                                                                               │
│  - delivery_problem                                                                                             │
│  - payment_issue                                                                                                │
│  - refund_request                                                                                               │
│  - display_issue                                                                                                │
│  - battery_life                                        

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Customer Support Ticket Classifier                                                                      │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  {                                                                                                              │
│    "ticket_type": "technical_issue",                                                                            │
│    "ticket_subject": "hardware_issue",                                                                          │
│    "reasoning": "The customer is experiencing a problem with their Dell XPS not turning on and not charging     │
│  properly, which indicates a hardware issue. This aligns with the technical_issue ticket type and               │
│  hardware_issue ticket subject."                                                                                │
│  }                                                                                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── Execution Traces ────────────────────────────────────────────────╮
│                                                                                                                 │
│  🔍 Detailed execution traces are available!                                                                    │
│                                                                                                                 │
│  View insights including:                                                                                       │
│    • Agent decision-making process                                                                              │
│    • Task execution flow and timing                                                                             │
│    • Tool usage details                                                                                         │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Would you like to view your execution traces? [y/N] (20s timeout): 

Exception in thread Thread-190 (get_input):
Traceback (most recent call last):
  File "C:\Users\40108102\AppData\Roaming\uv\python\cpython-3.13.5-windows-x86_64-none\Lib\threading.py", line 1043, in _bootstrap_inner
    self.run()
    ~~~~~~~~^^
  File "C:\work\crewai_customer_support_chatbot\deep_resarch\.venv\Lib\site-packages\ipykernel\ipkernel.py", line 788, in run_closure
    _threading_Thread_run(self)
    ~~~~~~~~~~~~~~~~~~~~~^^^^^^
  File "C:\Users\40108102\AppData\Roaming\uv\python\cpython-3.13.5-windows-x86_64-none\Lib\threading.py", line 994, in run
    self._target(*self._args, **self._kwargs)
    ~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\work\crewai_customer_support_chatbot\deep_resarch\.venv\Lib\site-packages\crewai\events\listeners\tracing\utils.py", line 359, in get_input
    response = input().strip().lower()
               ~~~~~^^
  File "C:\work\crewai_customer_support_chatbot\deep_resarch\.venv\Lib\site-packages\ipykernel\kernelbase.py", line 1475, in raw



Classification Result:
ticket_type='technical_issue' ticket_subject='hardware_issue' reasoning='The customer is experiencing a problem with their Dell XPS not turning on and not charging properly, which indicates a hardware issue. This aligns with the technical_issue ticket type and hardware_issue ticket subject.'


In [84]:
def fix_my_dataset_classification(df,rows_to_fix):
    df_c = df.copy()
    df_c = df.head(rows_to_fix)
    df_c['new_ticket_type'] = ''
    df_c['new_ticket_subject'] = ''
    df_c['reasoning'] = ''
    df_c.reset_index(drop=True,inplace=True)
    for index,row in df_c.iterrows():
        if index < rows_to_fix:
            print(f'fixing ticket {index}')
            ticket_description = row['ticket_description']
            result = classify_ticket(ticket_description)
            df_c.at[index,'new_ticket_type'] = result.ticket_type
            df_c.at[index,'new_ticket_subject'] = result.ticket_subject
            df_c.at[index,'reasoning'] = result.reasoning
    return df_c

In [85]:
new_df = fix_my_dataset_classification(df,10)
for index,row in new_df.iterrows():
    print(f'ticket {index} is {row["new_ticket_type"]} and {row["new_ticket_subject"]} old classifcation was {row["ticket_type"]} and {row["ticket_subject"]}')

fixing ticket 0


C:\Users\40108102\AppData\Local\Temp\ipykernel_46660\4151986769.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_c['new_ticket_type'] = ''
C:\Users\40108102\AppData\Local\Temp\ipykernel_46660\4151986769.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_c['new_ticket_subject'] = ''
C:\Users\40108102\AppData\Local\Temp\ipykernel_46660\4151986769.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead



╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Customer Support Ticket Classifier                                                                      │
│                                                                                                                 │
│  Task: Analyze the following customer support ticket description and classify it                                │
│          into one of the available ticket types.                                                                │
│                                                                                                                 │
│          Ticket Description:                                                                                    │
│          i'm facing a problem with my {Dell XPS}. the {Dell XPS} is not turning on. it was working fine until   │
│  yesterday, but now it doesn't respond.                                                                         │
│                                                                                                                 │
│  1.8.3 i really i'm using the original charger that came with my {Dell XPS}, but it's not charging properly.    │
│                                                                                                                 │
│          Available Ticket Types:                                                                                │
│          - technical_issue                                                                                      │
│  - billing_inquiry                                                                                              │
│  - cancellation_request                                                                                         │
│  - product_inquiry                                                                                              │
│  - refund_request                                                                                               │
│                                                                                                                 │
│          Available Ticket subject:                                                                              │
│          - network_problem                                                                                      │
│  - account_access                                                                                               │
│  - data_loss                                                                                                    │
│  - software_bug                                                                                                 │
│  - product_setup                                                                                                │
│  - product_recommendation                                                                                       │
│  - cancellation_request                                                                                         │
│  - product_compatibility                                                                                        │
│  - hardware_issue                                                                                               │
│  - delivery_problem                                                                                             │
│  - payment_issue                                                                                                │
│  - refund_request                                                                                               │
│  - display_issue                                                                                                │
│  - battery_life                                        

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Customer Support Ticket Classifier                                                                      │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  {                                                                                                              │
│    "ticket_type": "technical_issue",                                                                            │
│    "ticket_subject": "hardware_issue",                                                                          │
│    "reasoning": "The customer is experiencing a problem with their Dell XPS not turning on and not charging     │
│  properly, which indicates a hardware issue. This aligns with the technical_issue ticket type and               │
│  hardware_issue ticket subject."                                                                                │
│  }                                                                                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── Execution Traces ────────────────────────────────────────────────╮
│                                                                                                                 │
│  🔍 Detailed execution traces are available!                                                                    │
│                                                                                                                 │
│  View insights including:                                                                                       │
│    • Agent decision-making process                                                                              │
│    • Task execution flow and timing                                                                             │
│    • Tool usage details                                                                                         │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Customer Support Ticket Classifier                                                                      │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  {                                                                                                              │
│    "ticket_type": "technical_issue",                                                                            │
│    "ticket_subject": "software_bug",                                                                            │
│    "reasoning": "The customer is experiencing an unresolved issue with missing content on their Xbox, which     │
│  suggests a technical problem related to software functionality. The repeated contact with customer support     │
│  indicates a persistent software bug that has not been addressed."                                              │
│  }                                                                                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── Execution Traces ────────────────────────────────────────────────╮
│                                                                                                                 │
│  🔍 Detailed execution traces are available!                                                                    │
│                                                                                                                 │
│  View insights including:                                                                                       │
│    • Agent decision-making process                                                                              │
│    • Task execution flow and timing                                                                             │
│    • Tool usage details                                                                                         │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Would you like to view your execution traces? [y/N] (20s timeout): 

Exception in thread Thread-216 (get_input):
Traceback (most recent call last):
  File "C:\Users\40108102\AppData\Roaming\uv\python\cpython-3.13.5-windows-x86_64-none\Lib\threading.py", line 1043, in _bootstrap_inner
    self.run()
    ~~~~~~~~^^
  File "C:\work\crewai_customer_support_chatbot\deep_resarch\.venv\Lib\site-packages\ipykernel\ipkernel.py", line 788, in run_closure
    _threading_Thread_run(self)
    ~~~~~~~~~~~~~~~~~~~~~^^^^^^
  File "C:\Users\40108102\AppData\Roaming\uv\python\cpython-3.13.5-windows-x86_64-none\Lib\threading.py", line 994, in run
    self._target(*self._args, **self._kwargs)
    ~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\work\crewai_customer_support_chatbot\deep_resarch\.venv\Lib\site-packages\crewai\events\listeners\tracing\utils.py", line 359, in get_input
    response = input().strip().lower()
               ~~~~~^^
  File "C:\work\crewai_customer_support_chatbot\deep_resarch\.venv\Lib\site-packages\ipykernel\kernelbase.py", line 1475, in raw

ticket 0 is technical_issue and hardware_issue old classifcation was technical_issue and network_problem
ticket 1 is technical_issue and software_bug old classifcation was billing_inquiry and account_access
ticket 2 is technical_issue and battery_life old classifcation was billing_inquiry and data_loss
ticket 3 is technical_issue and hardware_issue old classifcation was cancellation_request and data_loss
ticket 4 is technical_issue and hardware_issue old classifcation was product_inquiry and software_bug
ticket 5 is technical_issue and software_bug old classifcation was billing_inquiry and product_setup
ticket 6 is technical_issue and data_loss old classifcation was product_inquiry and account_access
ticket 7 is technical_issue and software_bug old classifcation was refund_request and software_bug
ticket 8 is technical_issue and hardware_issue old classifcation was technical_issue and product_recommendation
ticket 9 is technical_issue and software_bug old classifcation was cancellation

In [87]:
new_df.to_csv(r'C:\work\crewai_customer_support_chatbot\deep_resarch\data\customer_support_tickets_fixed.csv',index=False)